In [ ]:
# Ingredients to be used in 10 recipes

# ["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]
# ["bay scallops", "butter", "onions", "parsley", "lemons", "bread crumbs"]
# ["rice", "chicken", "cream of mushroom soup", "cream of chicken soup", "onion soup", "water"]
# ["coriander", "allspice", "cumin", "ground ginger", "turmeric", "cinnamon", "cayenne", "pepper", "salt", "lemon juice", "olive oil", "garlic", "chicken", "pita", "red onion", "parsley", "shredded red cabbage", "tahini"]
# ["sour cream", "heavy cream", "baking powder", "kosher salt", "sugar", "ground black pepper", "flour", "butter", "salt", "cutter"]
# ["hamburger", "egg", "bread crumbs", "onion", "milk", "salt", "pepper", "bacon", "catsup"]
# ["stew meat", "cabbage", "onion", "green bell peppers", "oil", "tomatoes", "garlic", "soy sauce"]
# ["chicken broth", "water", "chicken", "onion", "vegetables", "garlic", "salt", "fresh cilantro", "potatoes", "vegetable oil"]
# ["pork sparerib", "black bean sauce", "Chinese rice wine", "cornstarch", "ginger", "garlic", "ground black pepper", "cooking oil", "sesame oil", "sugar", "red chili pepper"]
# ["shell pasta", "red pepper", "green onion", "seafood", "peas", "olive oil", "sesame oil", "rice vinegar", "lemon", "mayonnaise"]

In [ ]:
# Importing the required libraries & modules
from langchain_community.document_loaders.csv_loader import CSVLoader


In [ ]:
import pyodbc
server = 'rag_application_server'
database_name = 'personalized_recipe_generator'
user = 'root'
password = 'root'
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database_name};UID={user};PWD={password}'
conn = pyodbc.connect(connection_string)

host="localhost"
port="3306"

In [ ]:
import pandas as pd

import os
current_path = os.getcwd()
print(current_path)

os.chdir('../dataset/')
path_base_dataset = os.getcwd()
print(path_base_dataset)

os.chdir('../exploratory_data_analysis/')
path_base_notebook = os.getcwd()
print(path_base_notebook)

NUM_DATA_SAMPLES = 50000
file_name = '/sampled_dataset_' + str(NUM_DATA_SAMPLES) + '.csv'
sampled_data_file_path = path_base_dataset + file_name

df = pd.read_csv(sampled_data_file_path)
df.head()

In [ ]:
df.to_sql(name='personalized_recipe_generator', con=conn, if_exists='replace', index=False)

In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import AIMessage, HumanMessage

def init_database(user: str, password: str, host: str, port: str, database: str) -> SQLDatabase:
  db_uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
  return SQLDatabase.from_uri(db_uri)

db = init_database(
  user="root",
  password="root",
  host="localhost",
  port="3306",
  database=database_name
  )

In [ ]:
print(db.dialect)
print(db.get_usable_table_names())